# **Exercício XGBoost**
---

Esse notebook é o estudo do algorítimo XGBoost e é uma reprodução do exercício resolvido pelo criador do algorítimo, Tong He no vídeo abaixo:

https://www.youtube.com/watch?time_continue=717&v=ufHo8vbk6g4&feature=emb_title

Os dados para a execução do exercício são de uma competição do Kaggle "Higgs Boson Machine Learning Challenge":

https://www.kaggle.com/c/higgs-boson


## **Exemplo com dataset do pacote**

### **Pacote e dados**

O pacote já conta com 2 datasets, um de treino e um de teste.

In [34]:
# Carregar pacote
library(xgboost)
library(methods)

In [12]:
# Carregar dataset
data(agaricus.train, package="xgboost")
data(agaricus.test, package="xgboost")
train = agaricus.train
test = agaricus.test

# Estrutura dos dados
str(train)

### **O algorítimo**

**Informações mínimas**

- `Dados de entrada` : dados em forma de matriz
- `Variável target` : Um vetor numérico, usar número começando de 0 para classificação.
- `Objetivo` : "reg:linear" ou "binary:logistic"
- `Número de interações` : número de árvores adicionadas ao modelo

**Comando para criar o modelo**

In [23]:
# Para medir os resultados com base no erro
bst = xgboost(data = train$data, label = train$label, nround = 2, objective = "binary:logistic")

[1]	train-error:0.000614 
[2]	train-error:0.001228 


In [24]:
# Para medir os resultados com base no AUC (area under the curve)
bst = xgboost(data = train$data, label = train$label, nround = 2, objective = "binary:logistic", eval_metric = "auc")

[1]	train-auc:0.999238 
[2]	train-auc:0.999238 


**Previsão dos resultados**

In [26]:
# Visualização do resultado da previsão
pred = predict(bst, test$data)
head(pred)

[1] 0.2582498 0.7433221 0.2582498 0.2582498 0.2576509 0.2750908

**Cross validation**

In [31]:
# Modelo utilizando cross validation (para garantir que não está tendo overffiting)
cv.res = xgb.cv(data = train$data, nfold = 5, label = train$label, nround = 2, objective = "binary:logistic", eval_metric = "auc")

[1]	train-auc:0.998857+0.000349	test-auc:0.998466+0.001319 
[2]	train-auc:0.999189+0.000468	test-auc:0.998661+0.001452 


In [32]:
# Visualização dos resultados
cv.res

##### xgb.cv 5-folds
 iter train_auc_mean train_auc_std test_auc_mean test_auc_std
    1      0.9988574  0.0003485034     0.9984664  0.001319016
    2      0.9991890  0.0004684822     0.9986608  0.001452350

## **Exercício - Higgs Boson Competition**

Os dados para a execução do exercício são de uma competição do Kaggle "Higgs Boson Machine Learning Challenge":

https://www.kaggle.com/c/higgs-boson


In [112]:
# Carregar dataset
dtrain = read.csv("data/training.csv", header = TRUE)
head(dtrain)

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,100000,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,41.928,...,2,67.435,2.150,0.444,46.062,1.240,-2.475,113.497,0.002653311,s
2,100001,160.937,68.768,103.235,48.146,-999.000,-999.000,-999.000,3.473,2.078,...,1,46.226,0.725,1.158,-999.000,-999.000,-999.000,46.226,2.233584487,b
3,100002,-999.000,162.172,125.953,35.635,-999.000,-999.000,-999.000,3.148,9.336,...,1,44.251,2.053,-2.028,-999.000,-999.000,-999.000,44.251,2.347388944,b
4,100003,143.905,81.417,80.943,0.414,-999.000,-999.000,-999.000,3.310,0.414,...,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000,5.446378212,b
5,100004,175.864,16.915,134.805,16.405,-999.000,-999.000,-999.000,3.891,16.405,...,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000,6.245332687,b
6,100005,89.744,13.550,59.149,116.344,2.636,284.584,-0.540,1.362,61.619,...,3,90.547,-2.412,-0.653,56.165,0.224,3.106,193.660,0.083414031,b


### **Criação do modelo**

In [113]:
# Mudar a variável target de character para 1 ou 0 (binário)
dtrain[33] = dtrain[33] == "s"

# Criar um dataframe só com os labels
label = as.numeric(dtrain[[33]])

# Colocar todos os outros dados no dataset data
data = as.matrix(dtrain[2:31])

# Mudança feita por razões da competição
testsize = 550000
weight = as.numeric(dtrain[[32]]) * testsize / length(label)
sumwpos = sum(weight * (label == 1.0))
sumwneg = sum(weight * (label == 0.0))

# Construção de uma matrix (os valores missing estão marcados com -999 no dataframe)
xgmat = xgb.DMatrix(data, label = label, weight = weight, missing = -999.0)

# Visualização dos dados
str(xgmat)

Class 'xgb.DMatrix' <externalptr> 
 - attr(*, ".Dimnames")=List of 2
  ..$ : NULL
  ..$ : chr [1:30] "DER_mass_MMC" "DER_mass_transverse_met_lep" "DER_mass_vis" "DER_pt_h" ...


In [ ]:
# Set dos parametros
param = list("objective" = "binary:logitraw",
            "scale_pos_weight" = sumwneg/sumwpos,
            "bst:eta" = 0.1,
            "bst:max_depth" = 6,
             "eval_metric" = "auc",
             "eval_metric" = "ams@0.15",
             "silent" = 1,
             "nthread" = 16)

# Criação do modelo
bst = xgboost(params = param, data = xgmat, nround = 120)

### **Aplicação do modelo**

In [121]:
# Dados para teste
dtest = read.csv("data/test.csv", header = TRUE)
data = as.matrix(dtest[2:31])

# Construção da matriz legível para o algorítimo
xgmat = xgb.DMatrix(data, missing = -999.0)

# Aplicação do modelo nos dados
ypred = predict(bst, xgmat)

# Formatação da tabela de saída
idx = dtest[[1]]
rorder = rank(ypred, ties.method = "first" )
threshold = 0.15
ntop = length(rorder) - as.integer(threshold * length(rorder))
plabel = ifelse(rorder > ntop, "s", "b")
outdata = list("EventId" = idx,
              "RankOrder" = rorder,
               "Class" = plabel)

# Salvar dados em csv
write.csv(outdata, file = "./data/submission.csv", quote=FALSE, row.names = FALSE)

# Visualização dos dados
dados = as.data.frame(outdata)
head(dados)

,EventId,RankOrder,Class
,<int>,<int>,<fct>
1,350000,16705,b
2,350001,216737,b
3,350002,348612,b
4,350003,486482,s
5,350004,134241,b
6,350005,184578,b
